In [132]:
import sys
import math
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install texttable

In [133]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from texttable import Texttable

df = pd.read_csv('/Users/sanjaychari/Documents/RPI/Tachyon_Research/ganglia_database/rrds/Manager/novadaq-far-datadisk-02.fnal.gov/novadaq-far-datadisk-02.fnal.gov.csv')
df.fillna(0, inplace=True)

zero_cols = df.columns[(df == 0).all()]
df.drop(labels=zero_cols, axis=1, inplace=True)

df_train = df[:int(0.8*len(df))]
df_test = df[int(0.8*len(df)):]

In [117]:
X = df.drop(columns=['Epoch Timestamp', 'FTS failed files', 'FTS rawA failed files', 'FTS rawB failed files'])
y = df['FTS failed files'].replace(26.0, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [118]:
clf = linear_model.SGDClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [119]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.92      0.96    110826
         1.0       0.86      1.00      0.92     55063

    accuracy                           0.95    165889
   macro avg       0.93      0.96      0.94    165889
weighted avg       0.95      0.95      0.95    165889



In [129]:
orig_coef_dic = {}
abs_coef_dic = {}

for i in range(len(X.columns)):
    orig_coef_dic[X.columns[i]] = clf.coef_[0][i]
    abs_coef_dic[X.columns[i]] = abs(clf.coef_[0][i])

most_significant_coefs = sorted(abs_coef_dic.items(), key=lambda item: -item[1])[:10]
signed_most_sig_coefs = []

for ele in most_significant_coefs:
    signed_most_sig_coefs.append([ele[0], orig_coef_dic[ele[0]]])

[['boottime', -5338859.772522029], ['swap_total', -126831.52905567957], ['swap_free', -126829.18262772872], ['mem_total', -121776.96907179453], ['mem_free', -55392.12463517025], ['DL DataBlock Size Rate', -38198.002363399704], ['DL DataAtom Size Rate', -38143.99024830519], ['mem_cached', -31752.86653436711], ['mem_buffers', -13118.613039720185], ['DL DataAtom Size', -152.61669144066911]]


In [135]:
print("Most Influential Fields on FTS Failed Files")
print("-"*50)
table = Texttable()
signed_most_sig_coefs.insert(0, ['Field', 'SGD Coefficient'])
table.add_rows(signed_most_sig_coefs)
print(table.draw())

Most Influential Fields on FTS Failed Files
--------------------------------------------------
+------------------------+-----------------+
|         Field          | SGD Coefficient |
+========================+=================+
| boottime               | -5338859.773    |
+------------------------+-----------------+
| swap_total             | -126831.529     |
+------------------------+-----------------+
| swap_free              | -126829.183     |
+------------------------+-----------------+
| mem_total              | -121776.969     |
+------------------------+-----------------+
| mem_free               | -55392.125      |
+------------------------+-----------------+
| DL DataBlock Size Rate | -38198.002      |
+------------------------+-----------------+
| DL DataAtom Size Rate  | -38143.990      |
+------------------------+-----------------+
| mem_cached             | -31752.867      |
+------------------------+-----------------+
| mem_buffers            | -13118.613      |
+----